# Plant Disease Identification by using Computer Vision

## 1. Introduction

Plants are the foundation of life on Earth, providing the oxygen we breathe, the food we eat, and countless other resources vital to our survival. However, plants, like all living organisms, are susceptible to diseases that can reduce their health, vitality, and productivity. Early detection and classification of these diseases can play a critical role in ensuring food security and maintaining ecological balance.

With the advent of technology, particularly deep learning, we can now automate the process of detecting and classifying plant diseases. This project aims to build a deep learning model that can identify various plant diseases from images, enabling quicker response times and potentially saving crops.

### Objectives
**Data Preprocessing:** Before training our model, the images need to be preprocessed. This includes resizing, normalization, and data augmentation, ensuring the model is exposed to a variety of disease manifestations.

**Model Building:** We utilize a custom model, ResNet9, tailored for this classification task. Deep neural networks like ResNet have been revolutionary in image classification tasks due to their ability to learn hierarchical features from images.

**Training & Validation:** Training a model requires feeding it data and iteratively adjusting it to minimize errors. We'll also validate our model's performance on unseen data to ensure it generalizes well.

**Evaluation:** Beyond accuracy, we'll delve into other metrics like precision, recall, and F1-score. Additionally, we'll visualize our model's predictions using a confusion matrix to understand where it might be making mistakes.

### Dataset
The dataset comprises images of various plants, both healthy and affected by a variety of diseases. Our challenge is to distinguish between these different classes based on visual patterns. Each class is organized in a folder, making it easier to preprocess and load the data.

### Applications
Once fine-tuned, such a model could be incorporated into mobile applications for farmers or botanists. By simply taking a photo of a plant leaf, they could instantly determine if the plant is healthy or identify a potential disease, allowing for timely interventions.



## 2. Imports & Setup
Change the line below to reflect the actual working directory.

In [23]:
import os

os.chdir("/Users/davitzi/Projects/plantDiseasesRecognition/")
!pwd

/Users/davitzi/Projects/plantDiseasesRecognition


In [37]:
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from models import ResNet9
from plots import plot_loss_and_accuracy, plot_confusion_matrix, compute_metrics, plot_metrics
from preprocess import get_dataloaders_and_classes
from src.utils import print_diseases, print_data_frame
from tqdm import tqdm

import random

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms

import torch.nn as nn
import torch.nn.functional as F

We'll also be using utility functions to print diseases and data frame summaries.



In [25]:
from src.utils import print_diseases, print_data_frame


## 3. Dataset Overview

Before diving into the model training, it's crucial to understand the dataset we're working with.

In [26]:
print_diseases()


Unique plants: 
{'Squash', 'Orange', 'Soybean', 'Grape', 'Corn_(maize)', 'Peach', 'Cherry_(including_sour)', 'Tomato', 'Pepper,_bell', 'Raspberry', 'Potato', 'Apple', 'Strawberry', 'Blueberry'}
Total number: 14
Unique diseases: 
{'Tomato_mosaic_virus', 'Cedar_apple_rust', 'Haunglongbing_(Citrus_greening)', 'Cercospora_leaf_spot Gray_leaf_spot', 'Common_rust_', 'Esca_(Black_Measles)', 'Early_blight', 'Late_blight', 'Spider_mites Two-spotted_spider_mite', 'Northern_Leaf_Blight', 'Leaf_scorch', 'Powdery_mildew', 'Tomato_Yellow_Leaf_Curl_Virus', 'Black_rot', 'Leaf_blight_(Isariopsis_Leaf_Spot)', 'Target_Spot', 'Septoria_leaf_spot', 'Leaf_Mold', 'Apple_scab', 'Bacterial_spot'}
Total number: 20


This provides a quick overview of the unique plants and diseases in our dataset.

We can see there are 14 total number of various plants as well as 20 different type of diseases, some of them sharred in common, among our classes.

In [35]:
print_data_frame()


                                              Disease  No. of Images
0                                Strawberry___healthy           1824
1                                   Grape___Black_rot           1888
2                               Potato___Early_blight           1939
3                                 Blueberry___healthy           1816
4                              Corn_(maize)___healthy           1859
5                                Tomato___Target_Spot           1827
6                                     Peach___healthy           1728
7                                Potato___Late_blight           1939
8                                Tomato___Late_blight           1851
9                        Tomato___Tomato_mosaic_virus           1790
10                             Pepper,_bell___healthy           1988
11           Orange___Haunglongbing_(Citrus_greening)           2010
12                                 Tomato___Leaf_Mold           1882
13         Grape___Leaf_blight_(Is

By examining our data distribution, we can make informed decisions on preprocessing techniques, model selection, and evaluation metrics.


## 4. Data Preprocessing

In deep learning, raw data is rarely in the right format or structure for training. Our preprocessing involves normalization, augmentation and arranging the data into a more convenient structure.

Here's the preprocessing approach we're taking:

In [38]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [39]:
def get_dataloaders_and_classes(batch_size=32, num_samples_per_class=500):
    train_dataset = datasets.ImageFolder(root='data/train', transform=train_transform)

    # Gathering indices per class
    indices_per_class = {}
    for idx, (_, class_idx) in enumerate(train_dataset):
        if class_idx not in indices_per_class:
            indices_per_class[class_idx] = []
        indices_per_class[class_idx].append(idx)

    # Limiting number of samples per class
    limited_indices = []
    for class_idx, indices in indices_per_class.items():
        limited_indices.extend(random.sample(indices, min(num_samples_per_class, len(indices))))

    # Split the limited indices into train and test indices
    train_idx, val_idx = train_test_split(limited_indices, test_size=0.2,
                                          stratify=[train_dataset[i][1] for i in limited_indices], random_state=42)

    # Creating data loaders
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)

    test_set = datasets.ImageFolder(root='data/valid', transform=val_test_transform)
    test_loader = DataLoader(test_set, batch_size=batch_size)

    num_classes = len(train_dataset.classes)

    return train_loader, val_loader, test_loader, num_classes

In [41]:
# Instantiate our dataloaders with specific transformations
train_loader, val_loader, test_loader, num_classes = get_dataloaders_and_classes()


## 5. Model Training

Our choice of model is ResNet9. ResNet architectures are well-suited for image classification tasks due to their deep architectures and skip connections.

### ResNet9 Model for Plant Disease Classification

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pool=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.pool = pool

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x, inplace=True)
        if self.pool:
            x = F.max_pool2d(x, 4)
        return x


class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ResNet9, self).__init__()

        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True)
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))

        self.conv3 = ConvBlock(128, 256, pool=True)
        self.conv4 = ConvBlock(256, 512, pool=True)
        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))

        self.classifier = nn.Sequential(nn.AdaptiveAvgPool2d(1),
                                        nn.Flatten(),
                                        nn.Linear(512, num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

#### Overview:

The `ResNet9` model presented here is a modified and simplified version of the standard ResNet architectures. ResNets, or Residual Networks, have been a groundbreaking innovation in the deep learning community. Their main contribution is the introduction of "skip connections" or "residual connections" that allow gradients to flow through a network. This alleviates the vanishing gradient problem in very deep networks, enabling the training of deeper models.

#### **Model Architecture:**

1. **ConvBlock Module**:
    - This custom block is an encapsulation of the typical layers used in ConvNets:
        * **Conv2D**: A convolutional layer that applies filters to the input data.
        * **BatchNorm2D**: Batch normalization to stabilize and accelerate the training of deep networks.
        * **ReLU**: An activation function that introduces non-linearity.
        * **MaxPool2d** (optional): A pooling layer that reduces the spatial dimensions of the data.
    - This block improves code modularity and reduces redundancy.

2. **Convolutional and Residual Layers**:
    - `conv1`: This is a standard convolution block without pooling, mapping `in_channels` to 64 channels.
    - `conv2`: This block increases the channels from 64 to 128 and reduces spatial dimensions via pooling.
    - `res1`: The first residual block that consists of two convolution blocks. The output from `conv2` is added to the output of this block. This is the essence of the "residual connection".
    - `conv3`: A block that maps 128 channels to 256 with pooling.
    - `conv4`: A block that maps 256 channels to 512 with pooling.
    - `res2`: The second residual block, similar to `res1`. The output from `conv4` is added to the output of this block.

3. **Classifier**:
    - **AdaptiveAvgPool2d**: It reduces the spatial dimensions to 1x1. This is especially useful because irrespective of the input size, the output will always be of fixed size per batch (i.e., batch_size x 512 x 1 x 1).
    - **Flatten**: Converts the 3D output from the previous layer to 2D.
    - **Linear**: Fully connected layer that produces the final output with a size of `num_classes`.

#### **Forward Pass**:

Starting with an input `xb`:
- The input first passes through `conv1` and `conv2`.
- It then goes through the `res1` residual block and the result gets added to the output of `conv2`.
- The data is further passed through `conv3` and `conv4`.
- The output of `conv4` goes through the `res2` block and is added to the result from `conv4`.
- Finally, the classifier produces the output with a size of `num_classes`.

In [40]:
MODEL_DICT = {
    'ResNet9': ResNet9
}

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
def train_one_epoch(model, loader, criterion, optimizer, dev, epoch):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    for images, labels in tqdm(loader, desc=f"Training epoch {epoch + 1}", leave=False):
        images, labels = images.to(dev), labels.to(dev)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return running_loss / len(loader.dataset), correct / total

In [32]:
def validate_model(model, loader, criterion, dev, epoch):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(loader, desc=f"Validating epoch {epoch + 1}", leave=False):
            images, labels = images.to(dev), labels.to(dev)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(loader.dataset), correct / total

In [42]:
def train_model(model_class, num_epochs, num_samples_per_class, batch_size, learning_rate):
    if not os.path.exists('models'):
        os.makedirs('models', exist_ok=True)

    train_loader, val_loader, test_loader, num_classes = (
        get_dataloaders_and_classes(batch_size, num_samples_per_class))

    print_diseases()
    print_data_frame()

    model = model_class(3, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list = [], [], [], []

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch)
        val_loss, val_accuracy = validate_model(model, val_loader, criterion, device, epoch)

        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        train_accuracy_list.append(train_accuracy)
        val_accuracy_list.append(val_accuracy)

        print(
            f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, "
            f"Val Loss: {val_loss:.4f}, "
            f"Train Accuracy: {train_accuracy:.4f}, "
            f"Val Accuracy: {val_accuracy:.4f}")

        if (epoch + 1) % 5 == 0:  # Evaluate every 5 epochs
            plot_loss_and_accuracy(str(model_class.__name__),
                                   train_loss_list,
                                   val_loss_list,
                                   train_accuracy_list,
                                   val_accuracy_list)

            confusion_matrix = torch.zeros(num_classes, num_classes)
            true_labels = []
            predicted_labels = []
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
                    for t, p in zip(labels.view(-1), predicted.view(-1)):
                        confusion_matrix[t.long(), p.long()] += 1
                    true_labels.extend(labels.cpu().numpy())
                    predicted_labels.extend(predicted.cpu().numpy())

            precision, recall, f1 = compute_metrics(true_labels, predicted_labels)
            print(f"Epoch {epoch + 1}: Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

            plot_metrics(model_class, precision, recall, f1, epoch + 1)
            plot_confusion_matrix(model_class, confusion_matrix, val_loader.dataset.classes)


This function is designed to handle the complete training process of a neural network model. It includes loading the data, configuring the model, defining the loss criterion, optimizing the model, training the model for a specified number of epochs, and finally evaluating the model's performance on validation data.

#### Arguments:

- model name
- number of epochs
- number of samples per class
- number of samples per batch
- learning rate

In [ ]:
train_model('ResNet9', 10, 100, 20, 0.001)

## 6. Evaluation

Once our model is trained, it's imperative to evaluate its performance on unseen data to gauge its real-world applicability.

In [ ]:
model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

## 7. Conclusion

By the end of this notebook, we've preprocessed our dataset, trained a model, and evaluated its performance. The insights from this project can be used to refine the approach, choose different architectures, or implement real-world applications for plant disease detection.